# Utilisation des réseaux euclidiens dans pour le décodage de code de dimension 1

## Génération d'équations


On va essayer de décoder des équations de la forme : 

\begin{equation}
 y_i +C_ix + D_i \equiv 0 [p]
\end{equation}
    Que l'on peut reformuler en :
\begin{equation}
y_i + A_i y_0 + B_i \equiv 0 [p] , i = 1\dots n
\end{equation}

avec $y_0=x$ ou $y_0=y_h$

On va d'abord générer les différents $y_i$

In [8]:
prime_size = 10
h = 2
y = []
a = []
b = []
p = random_prime(2^prime_size-1, lbound=2^(prime_size-1))
F = GF(p)
x = F(randint(0,p-1))
for i in range(h):
    a.append(F(randint(0,p-1)))
    b.append(F(randint(0,p-1)))
    y.append(-a[i]*x - b[i])
    
len(y)

2

On va maintenant générer $n = h$ équations, les equations s'écrivent de la forme $y_i + A_ix + B_i \equiv 0 [p]$, c'est a dire si on connait certains bits $c_i$ : $(z_i + c_i) + A_ix + B_i \equiv 0 [p]$

In [35]:
v = vector(ZZ,h+1)
v[0] = 0
sample = []
arranged_sample = []
known_bytes = 10
mask = 0
for i in range(known_bytes):
    mask += 1<<i
z = vector(ZZ,h+1)
z[0] = x

for i in range(h):
    j = i+1
    sample.append(F(y[i].lift()+(a[i]*x).lift() + b[i].lift()))
    ci = y[i].lift()&mask
    zi = y[i].lift()-ci
    z[j] = zi
    v[j] = ci + b[i].lift()
    arranged_sample.append(z[j] + a[i]*z[0] + v[j])
    print(sample[i]==arranged_sample[i])
    


True
True


In [36]:
A = zero_matrix(ZZ,h+1)
A[0,0] = -1
for k in range(1,h+1):
    A[0,k] = a[k-1]
    A[k,k] = p

def Babai_LLL(A, target):
    B = A.LLL()
    G = B.gram_schmidt()[0]
    diff = target
    for i in reversed(range(B.nrows())):
        diff -=  B[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff
print(A.str()+ "\n" + str(a) + "\n" + str(p))

[ -1 427 667]
[  0 997   0]
[  0   0 997]
[427, 667]
997


In [37]:
w = Babai_LLL(A,v)
print(str((w-v)) + " , " + str(z) + "," + str(y))

(-21, -19, -11) , (544, 0, 0),[717, 789]


In [38]:
print(str(w) + "," + str(v))

(-21, 991, 1046),(0, 1010, 1057)


In [39]:
F(w[0]-v[0])

976

In [41]:
x

544